## 5.cuckoo

In [34]:
import boto3
import botocore
from botocore.exceptions import ClientError
import os, time, json, subprocess

import s3, iam, ec2, lambda_fn, eventbridge as event

In [35]:

ACCOUNT_ID = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION = os.environ['AWS_DEFAULT_REGION']
VPC_ID = 'vpc-03617a8a518caa526'
DEFAULT_SUBNET_IDS = ['subnet-0980ad10eb313405b', 'subnet-0de97821ddb8236f7', 'subnet-0a160fbe0fcafe373', 'subnet-0ca765b361e4cb186', 'subnet-0a972b05a5b162feb']
SUBNET_ID = DEFAULT_SUBNET_IDS[0]
DEFAULT_SECURITY_GROUP_ID = 'sg-07f4ccd7a5be677ea'

In [36]:
# Initialize boto3 clients
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
lambda_client = boto3.client('lambda')
logs_client = boto3.client('logs')
events_client = boto3.client('events')
iam_client = boto3.client('iam')
ses_client = boto3.client('ses', region_name='us-east-1')  # Choose your AWS region

#### Create IAM Role

In [ ]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
role_name = "gpc_cuckoo_role"

# Create the IAM role with the assume role policy document
role_arn = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)['Role']['Arn']


#### Create IAM Role Policy (SQS, S3, Logs Permissions)
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sqs:*"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "ses:*"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:*",
                "s3-object-lambda:*"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:*"
            ],
            "Resource": "*"
        }
    ]
}

policy_name = "sqs_s3_logs_policies"

# Attach the inline policy to the IAM role
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps(policy_document)
)
print(f"Policy {policy_name} attached to role {role_name}")


#### Create S3 Bucket

In [ ]:

bucket_name = "gpc-cuckoo-bkt"

# Create the S3 bucket
s3.create_s3_bucket(bucket_name)

s3.upload_folder_to_s3('./templates', bucket_name, s3_folder_path=None, region=REGION)

In [55]:
# # Upload templates to the S3 bucket
# subprocess.run(['aws', 's3', 'cp', './templates', f's3://{bucket_name}', '--recursive'])

#### Create Lambda Function

In [56]:
lfn_name = "gpc_cuckoo"
zip_file = "./package.zip"  # Change this to the actual zip file path

# Create Lambda function
with open(zip_file, 'rb') as f:
    zipped_code = f.read()

lambda_arn = lambda_client.create_function(
    FunctionName=lfn_name,
    Runtime='python3.9',
    Role=role_arn,
    Handler='cuckoo.handler',
    Code={'ZipFile': zipped_code},
    Timeout=60,
    Environment={
        'Variables': {
            'foo': 'BAR'
        }
    }
)['FunctionArn']

#### Create CloudWatch Log Group

In [ ]:
log_group_name = "/aws/lambda/gpc_cuckoo"
logs_client.create_log_group(logGroupName=log_group_name)
logs_client.put_retention_policy(logGroupName=log_group_name,retentionInDays=7)

#### Create CloudWatch Events Rules

In [ ]:
cron_expressions = {
    "come_to_work": "cron(0 12 ? * MON-FRI *)",
    "daily_tasks": "cron(0 17 ? * MON-FRI *)",
    "pickup": "cron(0 22 ? * MON-FRI *)"
}

rules = {}
for rule_name, cron_expression in cron_expressions.items():
    rule_response = events_client.put_rule(Name=rule_name, ScheduleExpression=cron_expression)
    rules[rule_name] = rule_response['RuleArn']
    print(f"Event rule {rule_name} created with cron {cron_expression}")

# Add Targets to Event Rules (Invoke Lambda)
for rule_name, rule_arn in rules.items():
    events_client.put_targets(Rule=rule_name,Targets=[{'Id': f"{rule_name}_target",'Arn': lambda_arn}])
    print(f"Target added to rule {rule_name}")

#### Add Lambda Permissions for Event Rules

In [ ]:
for i, (rule_name, rule_arn) in enumerate(rules.items(), start=1):
    lambda_client.add_permission(
        FunctionName=lfn_name,
        StatementId=str(i),
        Action='lambda:InvokeFunction',
        Principal='events.amazonaws.com',
        SourceArn=rule_arn
    )
    print(f"Permission added for {rule_name} to invoke {lfn_name}")

print("All resources created successfully.")


#### Test The Lambda Function locally

- Create identities using AWS SES Service

In [67]:
emails = ['AMominNJ@gmail.com', 'bbcredcap3@gmail.com', 'A.Momin.NYC@gmail.com']
# Create an email identity
[ses_client.verify_email_identity(EmailAddress=email) for email in emails]

-   With the events created make sure to modify your code skeleton. Come back for these commands when you need to test locally
-   Testing the function from the command line: (Make sure you're in the directory with the cuckoo.py file)
-   Call the handler function and give it a sample event. This event can either be the event ARN text or something else that contains text that contains some of the text that the function is looking for. The text should contain `come_to_work` or `daily_tasks` or `pickup`. We'll also put them into the following format to match the way they will be passed to the function by the AWS event

In [ ]:
import cuckoo
cuckoo.handler({'resources':['some text containing the word pickup']},'context')
cuckoo.handler({'resources':['arn:aws:events:us-east-1:444510800003:rule/come_to_work']}, 'context')
cuckoo.handler({'resources':['a great daily_tasks test']},'context')

#### Delete Resources

In [ ]:
bucket = s3_resource.Bucket(bucket_name)

# Delete all objects in the bucket
bucket.objects.all().delete()

# Delete all object versions (if versioning is enabled)
# bucket.object_versions.all().delete()

# Finally, delete the bucket
bucket.delete()


In [ ]:
lambda_client.delete_function(FunctionName="gpc_cuckoo")

In [ ]:
event.delete_event_bridge_rule(list(cron_expressions.keys())[0])
event.delete_event_bridge_rule(list(cron_expressions.keys())[1])
event.delete_event_bridge_rule(list(cron_expressions.keys())[2])

In [ ]:
# Delete the log group
logs_client.delete_log_group(logGroupName=log_group_name)

In [ ]:
### DELETE IAM ROLE AT THE END AFTER DELETING ALL OTHER RESOURCES.
iam.delete_iam_role(role_name)

In [ ]:
[ses_client.delete_identity(Identity=identity) for identity in emails]